In [2]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [3]:
# Used to securely store your API key
from google.colab import userdata

In [5]:
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.

genai.configure(api_key='GEMINI_API_KEY')

In [6]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-pro
models/gemini-pro-vision


In [7]:
model = genai.GenerativeModel('gemini-pro')

In [8]:
%%time
response = model.generate_content("What is the meaning of life?")

CPU times: user 153 ms, sys: 13.4 ms, total: 167 ms
Wall time: 10.9 s


In [10]:
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=glm.GenerateContentResponse({'candidates': [{'finish_reason': 4, 'index': 0, 'safety_ratings': [], 'token_count': 0, 'grounding_attributions': []}], 'prompt_feedback': {'safety_ratings': [{'category': 9, 'probability': 1, 'blocked': False}, {'category': 8, 'probability': 1, 'blocked': False}, {'category': 7, 'probability': 1, 'blocked': False}, {'category': 10, 'probability': 1, 'blocked': False}], 'block_reason': 0}}),
)


In [11]:
response.prompt_feedback

safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HARASSMENT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_DANGEROUS_CONTENT
  probability: NEGLIGIBLE
}

Gemini can generate multiple possible responses for a single prompt. These possible responses are called `candidates`, and you can review them to select the most suitable one as the response.

View the response candidates with <a href="https://ai.google.dev/api/python/google/ai/generativelanguage/GenerateContentResponse#candidates"><code>GenerateContentResponse.candidates</code></a>:

In [12]:
response.candidates

[finish_reason: RECITATION
index: 0
]

By default, the model returns a response after completing the entire generation process. You can also stream the response as it is being generated, and the model will return chunks of the response as soon as they are generated.

To stream responses, use <a href="https://ai.google.dev/api/python/google/generativeai/GenerativeModel#generate_content"><code>GenerativeModel.generate_content(..., stream=True)</code></a>.

In [14]:
%%time
response = model.generate_content("What is the meaning of life?", stream=True)

CPU times: user 95.7 ms, sys: 10.4 ms, total: 106 ms
Wall time: 6.93 s


In [15]:
for chunk in response:
  print(chunk.text)
  print("_"*80)

The meaning of life is a deeply personal and philosophical question that has been pondered by
________________________________________________________________________________
 individuals and civilizations for centuries. There is no definitive answer, as the meaning of life can vary significantly depending on individual beliefs, values, and experiences.

Some
________________________________________________________________________________
 common approaches to understanding the meaning of life include:

1. **Purpose or Goal-Oriented Perspective:** This approach suggests that life has an inherent purpose or goal, often related to personal growth, making a difference in the world, or achieving a specific outcome.

2. **Experience-Oriented Perspective:** This perspective emphasizes
________________________________________________________________________________
 the importance of living a fulfilling and meaningful life through experiences that bring joy, connection, and personal satisfact

When streaming, some response attributes are not available until you've iterated through all the response chunks. This is demonstrated below:

In [18]:
response = model.generate_content("What is the meaning of life?", stream=True)

BadRequest: 400 Unknown error trying to retrieve streaming response. Please retry with `stream=False` for more details.

The `prompt_feedback` attribute works:

In [19]:
response.prompt_feedback

safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HARASSMENT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_DANGEROUS_CONTENT
  probability: NEGLIGIBLE
}

But attributes like <code>text</code> do not:

In [20]:
try:
  response.text
except Exception as e:
  print(f'{type(e).__name__}: {e}')

In [22]:
model = genai.GenerativeModel('gemini-pro')
chat = model.start_chat(history=[])
chat

ChatSession(
    model=genai.GenerativeModel(
        model_name='models/gemini-pro',
        generation_config={},
        safety_settings={},
        tools=None,
    ),
    history=[]
)

In [29]:
response = chat.send_message("Return Answer in json format, you are a recruiter at MNC, ask five questions and give answers for data analyst role in a company.")
to_markdown(response.text)

> ```json
> {
>   "questions": [
>     {
>       "question": "What is your understanding of the role of a data analyst?",
>       "answer": "A data analyst is responsible for collecting, cleaning, and analyzing data to identify trends and patterns. They use this information to make recommendations and solve business problems."
>     },
>     {
>       "question": "What are the key skills and qualifications for a data analyst?",
>       "answer": "Key skills and qualifications for a data analyst include:
> 
> * Strong analytical skills
> * Proficient in data analysis tools and techniques
> * Excellent communication and presentation skills
> * Ability to work independently and as part of a team
> * Bachelor's or Master's degree in a quantitative field, such as statistics, computer science, or business analytics"
>     },
>     {
>       "question": "What are the different types of data analysis?",
>       "answer": "There are many different types of data analysis, including:
> 
> * Descriptive analytics: This type of analysis describes the data and identifies trends and patterns.
> * Predictive analytics: This type of analysis uses data to predict future outcomes.
> * Prescriptive analytics: This type of analysis uses data to recommend actions to take.
> * Machine learning: This type of analysis uses algorithms to identify patterns in data and make predictions."
>     },
>     {
>       "question": "What are the challenges of working as a data analyst?",
>       "answer": "Some of the challenges of working as a data analyst include:
> 
> * Dealing with large and complex datasets
> * Cleaning and preparing data for analysis
> * Interpreting the results of analysis
> * Communicating findings to non-technical audiences"
>     },
>     {
>       "question": "What is the future of data analytics?",
>       "answer": "The future of data analytics is bright. As the amount of data in the world continues to grow, the demand for data analysts will only increase. Data analysts will play a key role in helping businesses to make sense of their data and make better decisions."
>     }
>   ]
> }
> ```

In [24]:
chat.history

[parts {
   text: "In one sentence, explain how a computer works to a young child."
 }
 role: "user",
 parts {
   text: "A computer is like a super smart box that remembers what we tell it to do and then follows our instructions."
 }
 role: "model"]

You can keep sending messages to continue the conversation. Use the `stream=True` argument to stream the chat:

In [27]:
response = chat.send_message("Okay, how about a more detailed explanation to a high schooler?")

for chunk in response:
  print(chunk.text)
  print("_"*80)

A computer is an electronic device that can be programmed to carry out a set of instructions. The basic components of a computer are the processor, memory, storage, input devices, and output devices. The processor is the "brain" of the computer and controls all of its operations. Memory stores the instructions and data that the computer is currently working on. Storage devices, such as hard drives and solid-state drives, store data that is not currently being used by the computer. Input devices, such as keyboards and mice, allow the user to interact with the computer. Output devices, such as monitors and printers, allow the computer to communicate with the user.

When you give a computer a command, the processor fetches the instructions for that command from memory. The processor then executes the instructions, which may involve performing calculations, accessing data from storage, or sending output to a device. Once the instructions have been executed, the computer returns to waiting 

In [28]:
for message in chat.history:
  display(to_markdown(f'**{message.role}**: {message.parts[0].text}'))

> **user**: In one sentence, explain how a computer works to a young child.

> **model**: A computer is like a super smart box that remembers what we tell it to do and then follows our instructions.

> **user**: Okay, how about a more detailed explanation to a high schooler?

> **model**: A computer is an electronic device that can be programmed to carry out a set of instructions. The basic components of a computer are the processor, memory, storage, input devices, and output devices. The processor is the "brain" of the computer and controls all of its operations. Memory stores the instructions and data that the computer is currently working on. Storage devices, such as hard drives and solid-state drives, store data that is not currently being used by the computer. Input devices, such as keyboards and mice, allow the user to interact with the computer. Output devices, such as monitors and printers, allow the computer to communicate with the user.
> 
> When you give a computer a command, the processor fetches the instructions for that command from memory. The processor then executes the instructions, which may involve performing calculations, accessing data from storage, or sending output to a device. Once the instructions have been executed, the computer returns to waiting for the next command.
> 
> Computers are used for a wide variety of tasks, including word processing, spreadsheets, presentations, games, and web browsing. They are also used in businesses, schools, and governments to manage data, control processes, and communicate with others.